# Exercise 5

- write code in modular, re-usable way

## Question 1: Cell and Verlet Lists

- key prerequisite to evaluate Particle-Particle interactions is fast access to neighboring particles 

### 1.a) Implement first a routine that creates cell lists from a set of particles in 1D/2D without symmetry.

For the analysis, each particle needs:
- position (1D, 2D, 3D)
- weights (how much baggage is it carrying)
- sometimes volume

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Generate 10000 particles in a grig from 0 to 1 in 2D grid of 0.05 length
n_particles = 10000
dims = 2
particle_pos = np.random.rand(n_particles, dims)
lower_bounds = np.array([0.0, 0.0])
upper_bounds = np.array([1.0, 1.0])
cell_side = 0.05 # 20 cells in each dimension  

__Cell List__: 
- each cell maintains list of its occupants
- construction of these lists for N particles is O(N)


In [ ]:
def create_cell_list(particle_pos, lower_bounds, upper_bounds, cell_size):

    # set up grid
    n_particles, dims = particle_pos.shape # columns, rows
    num_cells = np.ceil((upper_bounds - lower_bounds) / cell_size).astype(int) # number of cells in each dimension

    # create particle matrix
    particle_matrix = np.zeros((n_particles, dims + 1), dtype=int)
    particle_matrix[:, :dims] = particle_pos # first 2 columns are the particle positions in 2D
    cell_list = [[] for _ in range(np.prod(num_cells))] # create a list of empty lists

    # assign particles to cells
    for i in range(n_particles):

        # find the cell index for each particle
        # floor needs to be used because the cell index is 0-based
        cell_index = np.floor((particle_pos[i] - lower_bounds) / cell_side).astype(int)

        # convert the cell index to a single index
        cell_list_index = np.ravel_multi_index(cell_index, dims=num_cells)

        # add the particle to the cell list
        particle_matrix[i, dims] = len(cell_list[cell_list_index])
        cell_list[cell_list_index].append(i)

        # store cell assignment in final column of particle matrix
        particle_matrix[i, dims] = cell_list_index
    return particle_matrix, cell_list, num_cells #particle matrix contains x, y, cell
